In [2]:
import pysam as ps
import numpy as np
import collections
import matplotlib.pyplot as plt
from numpy.core.defchararray import array, center
import pysam as ps
import numpy as np
from collections import defaultdict
from scipy.ndimage.measurements import label, standard_deviation
from scipy.stats.mstats_basic import kstest, normaltest
from sklearn.cluster import KMeans
import sys
from scipy.signal import savgol_filter
import math
from subprocess import call
import os.path
from utils import Gene, TSS, Point
from scipy import stats
from sklearn import svm
import sympy
import math
from math import e
from keras.models import Sequential
from keras import layers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers
import tensorflow as tf
import random
import math

dic = {'train_1':'./train_HK.bed',
        'train_0':'./train_CCR.bed',
        'test_1':'./test_HK.bed',
        'test_0':'./test_CCR.bed'
        }

HK = []
CCR = []
with open(dic['train_1'],'r') as f:
    for line in f:
        lll = []
        ll = np.array(line.strip().split('\t'))
        for l in ll:
            lll.append(round(float(l)))
        HK.append(lll)
with open(dic['train_0'],'r') as f:
    for line in f:
        lll = []
        ll = np.array(line.strip().split('\t'))
        for l in ll:
            lll.append(round(float(l)))
        CCR.append(lll)
r = min(len(HK),len(CCR))
perm = random.sample(range(len(HK[0:r]) + len(CCR[0:r])), len(HK[0:r]) + len(CCR[0:r]))
data = np.vstack((HK[0:r], CCR[0:r]))
label = np.array([1] * len(HK[0:r]) + [0] * len(CCR[0:r]))
data = data[perm]
label = label[perm]
train_num = int(len(data) * 0.8)
test_num = int(len(data) * 0.2)
train_data = data[0:train_num]
train_label = label[0:train_num]
test_data = data[train_num:(train_num + test_num)]
test_label = label[train_num:(train_num + test_num)]

2021-10-28 21:36:37.751243: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-28 21:36:37.751282: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


FileNotFoundError: [Errno 2] No such file or directory: './train_HK.bed'

In [16]:
# def LSTM_deep():
#     model = Sequential()
#     model.add(layers.Bidirectional(layers.LSTM(19, return_sequences=True,input_shape=(19,1))))
#     model.add(layers.Convolution1D(filters=100, kernel_size=3, input_shape=(19,1), activation='relu'))
#     model.add(layers.MaxPooling1D(pool_size=2, strides=2))
#     model.add(layers.Dropout(0.2))
#     model.add(layers.Flatten())
#     model.add(layers.Dense(750, activation='relu', kernel_regularizer=None, bias_regularizer=None))
#     model.add(layers.Dropout(0.2))
#     model.add(layers.Dense(300, activation='relu', kernel_regularizer=None, bias_regularizer=None))
#     model.add(layers.Dense(1, activation='sigmoid', kernel_regularizer=None, bias_regularizer=None))
#     return model

In [ ]:
# model = LSTM_deep()
# # sgd = optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])
# early_stopping = EarlyStopping(monitor='val_loss', patience=10)
# history = model.fit(train_data, train_label, epochs=3, batch_size=128, validation_split=0.1,
#                     callbacks=[EarlyStopping(patience=10), ModelCheckpoint(filepath='./' + 'weight.h5', save_best_only=True)],
#                     shuffle=True, verbose=2)
# print(model.summary())

In [3]:
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Sequential
from keras import layers

max_features = 100
maxlen = 50
batch_size = 32

(train_x, train_y),(test_x, test_y) = imdb.load_data(num_words=max_features)


In [4]:
model = Sequential()
# model.add(layers.Convolution1D(filters=100,kernel_size=10,input_shape=(66,5),activation='relu',strides=1,padding='same' ) )
# model.add(layers.MaxPooling1D(pool_size=2,strides=2) )
# model.add(layers.Dropout(0.2))
model.add(layers.Embedding(max_features, 33) )
model.add(layers.Bidirectional(layers.LSTM(33, return_sequences=True)))
model.add(layers.Convolution1D(filters=100, kernel_size=3, input_shape=(33,132), activation='relu',strides=1,padding='same'))
model.add(layers.MaxPooling1D(pool_size=2, strides=2))
# model.add(layers.Dropout(0.2))
# model.add(layers.Flatten())
# model.add(layers.Dense(750, activation='relu', kernel_regularizer=None, bias_regularizer=None))
# model.add(layers.Dropout(0.2))
# model.add(layers.Dense(300, activation='relu', kernel_regularizer=None, bias_regularizer=None))
model.add(layers.Dense(1, activation='sigmoid', kernel_regularizer=None, bias_regularizer=None))

input_train = sequence.pad_sequences(train_x, maxlen=maxlen)
input_test = sequence.pad_sequences(test_x, maxlen=maxlen)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())
history = model.fit(input_train[:1000], train_y[:1000],epochs=3,batch_size=2,validation_split=0.2)

2021-10-28 21:36:50.740604: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-10-28 21:36:50.740650: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-10-28 21:36:50.740671: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jiay-virtual-machine): /proc/driver/nvidia/version does not exist
2021-10-28 21:36:50.740841: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 33)          3300      
_________________________________________________________________
bidirectional (Bidirectional (None, None, 66)          17688     
_________________________________________________________________
conv1d (Conv1D)              (None, None, 100)         19900     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, None, 100)         0         
_________________________________________________________________
dense (Dense)                (None, None, 1)           101       
Total params: 40,989
Trainable params: 40,989
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3


2021-10-28 21:36:51.571451: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


400/400 [==============================] - 8s 15ms/step - loss: 0.6958 - acc: 0.5094 - val_loss: 0.6940 - val_acc: 0.5070
Epoch 2/3
 47/400 [==>...........................] - ETA: 4s - loss: 0.6847 - acc: 0.5485

KeyboardInterrupt: 

In [ ]:

# input_train = sequence.pad_sequences(train_x, maxlen=maxlen)
# input_test = sequence.pad_sequences(test_x, maxlen=maxlen)

# model = Sequential()
# model.add(layers.Embedding(max_features, 32))
# model.add(layers.SimpleRNN(32))
# model.add(layers.Dense(1,activation='sigmoid'))

# # print(model.summary())
# model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
# # early_stopping = EarlyStopping(monitor='val_loss', patience=10)
# # history = model.fit(input_train[:1000], train_y[:1000],epochs=3,batch_size=2,validation_split=0.2)

# # history = model.fit(train_data, train_label,epochs=3,batch_size=2,validation_split=0.2)